# Waves And Markers Download Test

Test Setup:
* Any Proteus Instrument 1-12 channels. - Running with a service.
* Scope connected to channel 1 and 3  of the module.

The test downloads wavforms to channels and markers and sets signal generator on. 
Run script and verify on scope theat all channels and markers generate square wave pulses.
System frequency can be set below (the default is the maximum - for each system.)



In [12]:
import os
import sys
srcpath = os.path.realpath('../../SourceFiles')
sys.path.append(srcpath)
#import pyte_visa_utils as pyte
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst
from tevisainst import TEVisaInst

In [13]:
%matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
import matplotlib.pyplot as plt


CLEAR_TEST = True #flag for additional clear segment test.

In [14]:
#conType="PXI" 
conType="LAN"

if conType in "LAN":
    ip_addr = "192.168.0.117"
    inst = TEVisaInst(ip_addr, port=5025)
    model = ''
else:
    admin = TepAdmin()
    slot_ids = admin.get_slot_ids()
    print("Trying to connect to slot: {}".format(slot_ids[0]))
    inst = admin.open_instrument(slot_id=slot_ids[0])

if inst is not None:
      idn_str = inst.send_scpi_query('*IDN?')
      model = inst.send_scpi_query(":SYST:iNF:MODel?")
      if idn_str:
          prnmsg = 'Connected to: ' + idn_str.strip()
          print(prnmsg)
else:
      print("Not connected")    


Connected to: Tabor Electronics,P9484M,000000000000,1.223.0 --slot#: 5,


In [15]:
# The paranoia-level is either 
# low (0), normal (1) or debug (2)
paranoia_level = 2

print ('Reset instrument ..')

# Reset the instrument
inst.send_scpi_cmd( '*CLS; *RST', paranoia_level)

inst.send_scpi_cmd( ':TRACe:DELete:ALL', paranoia_level)

print('Done')

Reset instrument ..
Done


In [28]:
# Get the model name
model = inst.send_scpi_query(":SYST:iNF:MODel?")
print("Model: " + model)

# Infer the natural DAC waveform format
if 'P9082' in model:
    dac_mode = 8
    FREQ=9e9
else:
    dac_mode = 16
    FREQ=2.5e9

print("DAC waveform format: {0} bits-per-point".format(dac_mode))

# Get number of channels
resp = inst.send_scpi_query(":INST:CHAN? MAX")
print("Number of channels: " + resp)
num_channels = int(resp)

# Get the maximal number of segments
resp = inst.send_scpi_query(":TRACe:SELect:SEGMent? MAX")
print("Max segment number: " + resp)
max_seg_number = int(resp)

# Get the available memory in bytes of wavform-data (per DDR):
resp = inst.send_scpi_query(":TRACe:FREE?")
arbmem_capacity = int(resp)
print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity))

max_dac = 2 ** dac_mode - 1
half_dac = max_dac / 2.0

Model: P9484M
DAC waveform format: 16 bits-per-point
Number of channels: 4
Max segment number: 65536
Available memory per DDR: 17,171,421,120 wave-bytes


## System Frequency 

In [29]:
cmd=":FREQ:RAST {}".format(FREQ)
inst.send_scpi_cmd(cmd)
resp = inst.send_scpi_query(":FREQ:RAST?")
freq = float(resp)
print ("DAC Generate Freq:{0}".format(freq))

DAC Generate Freq:2500000000.0


## Prepare wave-data and markers-data segments for all channels


In [30]:
# Build wave-data and markers-data for 12 segments

print('Build wave-data and markers-data segments for {} channels ..'.format(num_channels))

seglen = 2** 12 #10 #2 ** 20 # 1MB
num_cycles = [2 ** n for n in range(num_channels)]

waves = [ None for _ in range(num_channels)]
marks = [ None for _ in range(num_channels)]

    
if (dac_mode == 16) and int(FREQ) <= 2.5e9:
  seg_mark_bytes = seglen // 4
else:
  seg_mark_bytes = seglen // 8
  if dac_mode != 16:
      num_channels = int(num_channels//2)
    
    
for ii in range(num_channels):
    ncycles = num_cycles[ii]
    cyclelen = seglen / ncycles
    
    x = np.linspace(start=0, stop=seglen, num=seglen, endpoint=False)
    y = np.fmod(x, cyclelen)
    y = (y <= cyclelen / 2) * max_dac
    y = np.round(y)
    y = np.clip(y, 0, max_dac)
    
    if dac_mode == 16:
        waves[ii] = y.astype(np.uint16)
    else:
        waves[ii] = y.astype(np.uint8)
    
    
    del x, y
print('Done')   
 

Build wave-data and markers-data segments for 4 channels ..
Done


In [31]:
for ii in range(num_channels):
    ncycles = num_cycles[ii]
    #cyclelen = seglen / ncycles
    
    cycle_bytes = seg_mark_bytes/ncycles
    
    #if dac_mode == 16 and int(FREQ) <= 2.5e9:
    #    cycle_bytes = cyclelen // 4
    #else:
    #    cycle_bytes = cyclelen // 8
        
    x = np.linspace(
        start=0, stop=seg_mark_bytes, num=seg_mark_bytes, endpoint=False)
    ym = np.fmod(x, cycle_bytes)
    ym = (ym <= cycle_bytes / 2) * 255
    ym = np.round(ym)
    ym = np.clip(ym, 0, 255)
    marks[ii] = ym.astype(np.uint8)
    
    del x, ym

## Download wave-data and markers-data to segments

In [32]:
# Download segments

# data-type code: 'B' = uint8, 'H' = uint16
#wav_dat_type = 'H' if dac_mode == 16 else 'B'
#mrk_dat_type = 'B'
num_channels=4
for ii in range(num_channels):
    ichan = ii
    channb = ichan + 1
    segnum = ichan % 2 + 1
    wav = waves[ichan]
    mrk = marks[ichan]
    print('Download wave to segment {0} of channel {1}'.format(segnum, channb))
    
    # Select channel
    cmd = ':INST:CHAN {0}'.format(channb)
    inst.send_scpi_cmd( cmd, paranoia_level)
    
    # Define segment
    cmd = ':TRAC:DEF {0}, {1}'.format(segnum, seglen)
    inst.send_scpi_cmd( cmd, paranoia_level)
    
    # Select the segment
    cmd = ':TRAC:SEL {0}'.format(segnum)
    inst.send_scpi_cmd( cmd, paranoia_level)
    
    # Increase the timeout before writing binary-data:
    inst.timeout = 30000
    
    # Send the binary-data:
    inst.write_binary_data(':TRAC:DATA', wav)
        
    resp = inst.send_scpi_query(':SYST:ERR?')
    resp = resp.rstrip()
    
    if not resp.startswith('0'):
        print('ERROR: "{0}" after writing binary values'.format(resp))
        
    print('Download markers to segment {0} of channel {1}'.format(segnum, channb))
    
    # Increase the timeout before writing binary-data:
    inst.timeout = 10000
    
    
    # Send the binary-data with *OPC? added to the beginning of its prefix.
    inst.write_binary_data(':MARK:DATA',mrk)
    
 
    resp = inst.send_scpi_query(':SYST:ERR?')
    resp = resp.rstrip()
    if not resp.startswith('0'):
        print('ERROR: "{0}" after writing binary values'.format(resp))

    # Play the specified segment at the selected channel:
    cmd = ':SOUR:FUNC:MODE:SEGM {0}'.format(segnum)
    inst.send_scpi_cmd( cmd, paranoia_level)

    # Turn on the output of the selected channel:
    cmd = ':OUTP ON'
    inst.send_scpi_cmd( cmd, paranoia_level)
    
    # Turn on the markers of the selected channel    
    resp = inst.send_scpi_query(':MARK:SEL? MAX')
    print(f'Maxinun markers = {resp}')
    resp = resp.rstrip()
    markers_per_chan = int(resp)
    print(f'Markers_per_chan = {markers_per_chan}')
    for imarker in range(markers_per_chan):
        cmd = ':MARK:SEL {0}; :MARK:STAT ON'.format(imarker + 1)
        inst.send_scpi_cmd(cmd, paranoia_level)
    
    resp = inst.send_scpi_query(':SYST:ERR?')
    print(resp)

print()

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

#del waves
#del wav

Download wave to segment 1 of channel 1
Download markers to segment 1 of channel 1
Maxinun markers = 2
Markers_per_chan = 2
0, no error
Download wave to segment 2 of channel 2
Download markers to segment 2 of channel 2
Maxinun markers = 2
Markers_per_chan = 2
0, no error
Download wave to segment 1 of channel 3
Download markers to segment 1 of channel 3
Maxinun markers = 2
Markers_per_chan = 2
0, no error
Download wave to segment 2 of channel 4
Download markers to segment 2 of channel 4
Maxinun markers = 2
Markers_per_chan = 2
0, no error

0, no error


## (Optional) Clear Segments For Channel

In [15]:
if CLEAR_TEST:
  cmd = ':INST:CHAN 1'
  inst.send_scpi_cmd( cmd, paranoia_level)
  cmd = ':TRAC:ZERO:ALL'
  inst.send_scpi_cmd(cmd, paranoia_level)

In [11]:
if True: #CLEAR_TEST:
  cmd = ':INST:CHAN 1'
  inst.send_scpi_cmd( cmd, paranoia_level)
  cmd = ':TRAC:ZERO 1'
  inst.send_scpi_cmd( cmd, paranoia_level)

In [20]:
if CLEAR_TEST:
  cmd = ':INST:CHAN 3'
  inst.send_scpi_cmd( cmd, paranoia_level)
  cmd = ':TRAC:ZERO 2'
  inst.send_scpi_cmd( cmd, paranoia_level)

In [76]:
cmd = ':INST:CHAN 1'
inst.send_scpi_cmd( cmd, paranoia_level)
cmd = ':MARK:SEL {0}; :MARK:STAT OFF'.format(1)
inst.send_scpi_cmd(cmd, paranoia_level)

0

In [22]:
resp = inst.query(':SYST:ERR?')
print(resp)

AttributeError: 'TEVisaInst' object has no attribute 'query'

## (Optional) Select Specific Markers - Check ON/OFF

In [ ]:

#chanlist = [1,2]#4,8,12]

#imarker = 2
#for channb in chanlist:
#  if channb <= num_channels:
#    cmd = ':INST:CHAN {0}'.format(channb)
#    pyte.send_cmd(inst, cmd, paranoia_level)
#    cmd = ':MARK:SEL {0}; :MARK:STAT OFF'.format(imarker)
#    pyte.send_cmd(inst, cmd, paranoia_level)
    

In [ ]:
##Function is assumed to be in the range of 0-1
def buildWaveFromFunc(func, period, ncycles, seglen, dac_mode=16):

    #global  num_channels, max_seg_number ,arbmem_capacity
    #num_cycles = (1, 2) #, 4, 8)
    #waves = [ None for _ in range(1)]

    cyclelen = seglen / ncycles
    
    ## Building wave form in 16 bit format.
    if dac_mode == 16: 
        y = (np.zeros(seglen).astype('uint16')) 
    else: 
        y = (np.zeros(seglen).astype('uint8')) 
    max_dac = 2 ** dac_mode - 1
    half_dac = max_dac / 2.0
    
    duty = cyclelen//2

    x = np.linspace(start=0, stop=period*ncycles, num=seglen, endpoint=False)
    y = func(x) *max_dac
    y = np.round(y)
    
    
    y = np.fmod(x, cyclelen)
    y = (y <= duty) * max_dac
    y = np.round(y)
    y = np.clip(y, 0, max_dac)

    
    x = np.linspace(start=0, stop=seglen, num=seglen, endpoint=False)
    y = np.fmod(x, cyclelen)
    y = (y <= duty) * max_dac
    y = np.round(y)
    y = np.clip(y, 0, max_dac)

    return y
